In [ ]:
import sys
sys.path.insert(0,  "C:\\Users\\Paul Valcke\\Documents\\GitHub\\CHIMES" )
import chimes as chm

from IPython.display import display,HTML,Markdown
from IPython.display import IFrame
from itables import init_notebook_mode,options
options.columnDefs = [{"className": "dt-left", "targets": "_all"}]
options.classes="display nowrap compact"
options.scrollY="400px"
options.scrollCollapse=True
options.paging=False

init_notebook_mode(all_interactive=True)
%matplotlib widget

# Pandas display
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option("display.colheader_justify","left")

## Tomas Mazak's workaround
import plotly
import plotly.graph_objs as go
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML('<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'))

# TL:DR (Too long, didn't read)

Here is the resume of everything seen in this tutorial
```
hub=chm.Hub('modelname')      # CHIMES object that you can use to interact with a model
hub.get_summary()             # Basic general informations
hub.get_fieldsproperties()    # dataframe description of fields
hub.get_Network()             # network visualisation of the model
hub.get_presets()             # display a table of the presets
hub.get_supplements()         # display a table of the supplements
hub.get_dfields()             # dictionnary of all fields properties
hub.dfields_as_reverse_dict() # classified by eqtypes
hub.get_dvalues()             # dictionnary of time evolving values after a run
hub.dataframe()               # fields values as multiindex dataframe
hub.get_dimensions()          # return the fields that change tensor dimensions  
hub.get_multisectoral()       # return fields classed by their tensor dimensons 
hub.dflags                    # status of the model
hub.dfunc_order               # order of the functions resolution
hub.dmisc                     # miscelaneous information
hub.dmodel                    # What is read from the model file
```


Depending of your level, we recommend: 

* **users** should know `hub=chm.Hub('modelname')`, `hub.get_summary()`, `hub.plot()`, `hub.set_fields()` (basic use), `hub.dvalues()`
* **advanced user** should know `hub.get_Network() `, `hub.get_fieldsproperties()`, `hub.get_preset()`, `hub.set_fields()` (advanced use), `hub.dfields()`
* **modeler** should know `hub.get_supplements`, `hub.dflags`, `hub.dfunc_order`, 
* **power-user** should know `hub.dmodel`

# Exploring the Hub

The `Hub` class is a central object for the user, providing an interface to interact with a given model.
Given a model name, the `Hub` class loads the model file and retrieves its logics, presets, and associated values. 
It performs tasks such as :
* executing local models coupling operation in the `_model_MODELNAME.py` file, 
* identifying parameters, state variables, and differential variables, 
* finding their associated properties (definitions, units, values...), 
* determining an order of calculation.

Users can: 
* access fields for value modification, 
* the structure and properties in the model, 
* perform simulations, 
* create plots
* conduct deeper analysis
* save the model

In [ ]:
# Preliminary points
import chimes as chm
name = 'Lorenz_Attractor'
hub=chm.Hub(name,verb=False) # We load the model for the rest of the system

## Exploring the content of the model

a `hub` has many introspection methods that return formatted data, that can be called as `hub.get_[...]`

### Summary
In most situations, the method `get_summary()` should cover your needs. 
It covers the flags of the system, theiur preset names, and fields by category with their attributes. As we will see after, get_summary displays the last value of the simulation when a run has been done.

In [ ]:
hub.get_summary()

### Field properties

A dataframe version is also possible using `hub.get_fieldproperties`, when using itables it give you the possibility to do search in the system. 

In [ ]:
hub.get_fieldsproperties()

### Field properties, as a network

As we said, a model is a web of fields linked by equations. We can thus represent it as such. and interact with it. 
The `get_Network()` method allow the customization of the representation.

You can hover on each nodes to get more information about its properties and dependencies 

In [ ]:
hub.get_Network()

### See the presets and supplements

Presets are a set of value for the system given by the author of a model, that allow one specific run and the display of a specific behavior of the system (either quantitative or qualitative). They are often associated with one serie of plots that shows it. They can be loaded into the hub using `set_preset(presetname)`. To see the presets, use `hub.get_preset`, or read `hub.get_summary()` or even `chm.get_model_documentation`

Supplements are functions that ease the use of a specific model and thus associated to it, either for the analyses, setting fields, or plots. they can be accessed the same way using `hub.get_supplements` or `get_model_documentation`. they can be then used with `hub.supplements[supplementname](arguments_of_the_function)`

In [ ]:
presets = hub.get_presets('list') # Return a list of the presets
print('preset list',presets)
dpresets = hub.get_presets('dict') # Return a dictionary of the presets with theirdescription
print('preset_dict',dpresets)
hub.get_presets() # display a table of the presets

In [ ]:
lsupp = hub.get_supplements('list') # Return a list of the supplements
print('preset list',presets)
dsupp = hub.get_supplements('dict') # Return a dictionary of the supplements
print('preset dictionnary',presets)
hub.get_supplements() # display a table of the supplements

### dfields

dfields is where all the fields properties of the model, digested by the hub, are located. It is a dictionnary of a dictionnary architecture: the first level of dfields is fields names, then their attributes, such as their value, definition and so on. 

the keys are: 
* `eqtype` type of equation ['differential', 'statevar', 'None']
* `func` the python logic that is used to calculate the value
* `source_exp` A string version of the solved equation
* `com` a comment explaining the python function
* `definition` the definition of the field
* `units` the unit of the field ( years, humans, meters...) in latex formalism
* `symbol` a latex symbol that represent the field
* `group` a classification that helps looking
* `value` a numpy tensor that keep all values (see the corresponding section) 
* `isneeded` Is the field required to calculate the dynamics
* `size` a list of two field that are used to know the multisectoral size of this one
* `multisect` a flag that tell if the field need all its dimensions
* `kargs`  
* `args`  
* `minmax` 

When a new analysis is done with the hub, a new category will appear for each fields in `dfields`. 
We recommend building your own tools (plot, analyses, etc) with dfields as the input, you'll have everything you need !

In [ ]:
R = hub.get_dfields()
print('the available fields are :',R.keys())
x = R['x']
print('the keys of the field x are :',x.keys())

you can filter `get_dfields` to get only the fields that correspond to your criteria. 

In [ ]:
Rdiff = hub.get_dfields(eqtype=['differential']) # You only get differential equations
print('Eqtype differential',Rdiff.keys())
Rdiff2 = hub.get_dfields(eqtype='statevar') # You get state variables
print('Eqtype differential',Rdiff2.keys())

### Getting fields according to criterias

It can be useful to have directly a fields classification: `get_dfields_as_reverse_dict` is doing so. use `crit` to choose the classification type, and you can add criteria after

In [ ]:
Rclas =hub.dfields_as_reverse_dict(   # Return a dictionnary with the list of outputs for each input
                                   crit='units', # classified by units
                                   eqtype=['differential', 'statevar'] # only eqtype differential and statevar are kept
                                    ) # only eqtype differential and statevar are kept
for k in Rclas.keys():
    print(k,Rclas[k].keys())

In [ ]:
Rclas = hub.dfields_as_reverse_dict(crit='eqtype') # classified by eqtypes
for k in Rclas.keys():
    print(k,Rclas[k].keys())

## Multi-region, Multi-agent, Multi-sector dimensions 

This is described in details in `09_Multisectoral_Multiregion`. 
When you deal with multiple regions, coupling between dimensions, multiple agents, you need to see which fields have different tensorial dimensions:
* `hub.get_dimensions()`  Give the different dimensions used in the model 
* `hub.get_multisectoral()` Give the dimensions of each fields

In [ ]:
hub=chm.Hub('E-CHIMES')
hub.get_dimensions()

In [ ]:
hub.get_multisectoral()

## Getting values in most cases

If you do not need the multisectoral properties, you can get a dictionnary of values with `dvalue`. 
If you have multiple regions of parrallel system you can slice only the one you want with nx = index, region = index

In [ ]:
values = hub.get_dvalues(idx=0,         # By default idx=0
                         Region=0,      # By default region=0
                         params=False   # If True, return parameters too, otherwise only time evolving quantities
                         )
print('type',type(values))
for k,v in values.items():
    print(k,np.shape(v))

### Dataframe approach

You can also access the values as a dataframe, that reshapes the dfields structure.

In [ ]:
df = hub.dataframe().transpose()
df

## Decorators

Those varaiables are not shaped for user readability but they can still be accessed. Those are protected and thus cannot be modified directly. 
You can modify the element by adding _ before its name if you really know what you're doing

In [ ]:
hub.dflags # status of the model

In [ ]:
hub.dfunc_order # order of the functions resolution

In [ ]:
hub.dmisc # miscelaneous information

In [ ]:
hub.dmodel # What is read from the model file